In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

# dataloader example

This notebook shows how to use the dataloader with the tensorflow library.
We should probably wrap this in a PyTorch datamodule at some point.

In [2]:
from tensorflow import keras

model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"
keras_model = keras.models.load_model(model_path)

2023-03-21 05:13:35.516993: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 05:13:35.635749: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-21 05:13:36.763723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 05:13:36.763792: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

2023-03-21 05:13:37.702593: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-21 05:13:37.702622: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (acmiyaguchi-dev): /proc/driver/nvidia/version does not exist
2023-03-21 05:13:37.702897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import librosa

test_path = "../data/raw/birdclef-2023/train_audio/palfly2/XC563813.ogg"
y, sr = librosa.load(test_path, sr=48_000)
y, sr

(array([-2.8339085e-05, -3.2334145e-05, -2.6037342e-05, ...,
         5.1300867e-06,  1.7665586e-06, -7.7035065e-06], dtype=float32),
 48000)

In [8]:
from birdclef.data.datasets import AudioPCMDataSet
from birdclef.data.transforms import ToBirdNETEmbedding

audio_path = "../data/raw/birdclef-2023/train_audio"

dataset = AudioPCMDataSet(
    audio_path,
    # duration of the clip in seconds
    min_duration=10,
    # how much to slide between consecutive sequence elements
    window_step=1,
    # how much to slide between consecutive sequences
    seq_step=5,
    # transforms (in particular, to the embedding space)
    transforms=[ToBirdNETEmbedding(model=keras_model.model)],
)
batch = next(iter(dataset))
batch, batch.shape

(array([[1.5712982 , 0.42635736, 1.0132436 , ..., 1.5472673 , 0.80867696,
         0.74898386],
        [1.5236511 , 0.5448162 , 0.609984  , ..., 0.8903057 , 0.68575984,
         0.5812848 ],
        [1.3799995 , 0.5603645 , 0.5621645 , ..., 0.77269024, 0.5598088 ,
         0.35564625],
        ...,
        [1.6219438 , 0.615823  , 0.8171584 , ..., 0.9147644 , 1.0211197 ,
         1.2703916 ],
        [1.2679403 , 0.42659807, 0.71017295, ..., 0.6935778 , 0.68383   ,
         0.8716263 ],
        [1.4103328 , 0.41387966, 1.2248114 , ..., 0.75333804, 0.6153552 ,
         0.7482261 ]], dtype=float32),
 (10, 320))